In [1]:
%load_ext autoreload
%autoreload 2

import sys
import copy
import math
import pickle

sys.path.append("../../")
#sys.path.append("/home/rosa/DefmodEnv/lib/python3.7/site-packages/")

import numpy as np
import torch
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

dm.Utilities.set_compute_backend('torch')
device = 'cpu'

In [4]:
source_image = dm.Utilities.load_greyscale_image("/home/leander/diffeo/implicitmodules/data/images/cross_+.png")
target_image = dm.Utilities.load_greyscale_image("/home/leander/diffeo/implicitmodules/data/images/cross_x.png")
# source_image = dm.Utilities.load_greyscale_image("/home/leander/diffeo/implicitmodules/data/images/heart_a.png")
# target_image = dm.Utilities.load_greyscale_image("/home/leander/diffeo/implicitmodules/data/images/heart_b.png")

FileNotFoundError: [Errno 2] No such file or directory: '/home/leander/diffeo/implicitmodules/data/images/cross_+.png'

In [134]:
%matplotlib qt5
plt.subplot(1, 2, 1)
plt.imshow(source_image)

plt.subplot(1, 2, 2)
plt.imshow(target_image)

plt.show()

In [135]:
aabb = dm.Utilities.AABB(0., source_image.shape[0], 0., source_image.shape[1])

# Fill the unit square
points_density = 0.1
translation_pts = aabb.fill_uniform_density(points_density)
#sigma_translation = 3./points_count
# sigma_translation = 10./points_count
sigma_translation = 5.

translation = dm.DeformationModules.Translations(2, translation_pts.shape[0], sigma_translation, gd=translation_pts.to(device=device).requires_grad_())

rotation = dm.DeformationModules.LocalRotation(2, 20., gd=torch.tensor([[10., 10.]], requires_grad=False))
#rotation = dm.DeformationModules.LocalRotation(2, 25., gd=torch.tensor([[10., 10.]], requires_grad=True))


In [136]:
model = dm.Models.ModelImageRegistration(source_image, [rotation], dm.Attachment.EuclideanPointwiseDistanceAttachment(), fit_gd=[True], lam=100.)

In [137]:
shoot_method = 'euler'
shoot_it = 10

In [138]:
#fitter = dm.Models.ModelFittingGradientDescent(model, 1e-2, verbose=False)
fitter = dm.Models.ModelFittingScipy(model)
costs = fitter.fit([target_image], 100, options={'shoot_method': shoot_method, 'shoot_it': shoot_it})

Initial energy = 6577.890
Time: 1.373555368045345
Iteration: 1 
Total energy = 5263.73583984375 
Attach cost = 5149.90576171875 
Deformation cost = 113.83026123046875
Time: 2.5698571710381657
Iteration: 2 
Total energy = 5260.5615234375 
Attach cost = 5143.185546875 
Deformation cost = 117.37579345703125
Time: 2.7716205990873277
Iteration: 3 
Total energy = 5258.83984375 
Attach cost = 5141.94384765625 
Deformation cost = 116.8961181640625
Time: 2.9763946610037237
Iteration: 4 
Total energy = 5242.0634765625 
Attach cost = 5127.39892578125 
Deformation cost = 114.66456604003906
Time: 3.5936331890989095
Iteration: 5 
Total energy = 4865.02587890625 
Attach cost = 4761.685546875 
Deformation cost = 103.34033966064453
Time: 3.795696455053985
Iteration: 6 
Total energy = 4708.55322265625 
Attach cost = 4601.1259765625 
Deformation cost = 107.42744445800781
Time: 4.200875126989558
Iteration: 7 
Total energy = 4584.130859375 
Attach cost = 4459.07763671875 
Deformation cost = 125.05312347412

Time: 13.080531920073554
Iteration: 39 
Total energy = 1022.1751098632812 
Attach cost = 909.4888916015625 
Deformation cost = 112.68622589111328
Time: 13.27845680899918
Iteration: 40 
Total energy = 1018.4186401367188 
Attach cost = 907.469970703125 
Deformation cost = 110.94866943359375
Time: 13.484630312072113
Iteration: 41 
Total energy = 1011.0225219726562 
Attach cost = 908.0103759765625 
Deformation cost = 103.01213073730469
Time: 13.881726824911311
Iteration: 42 
Total energy = 1010.0045166015625 
Attach cost = 910.9288940429688 
Deformation cost = 99.07565307617188
Time: 14.084572795080021
Iteration: 43 
Total energy = 1007.8017578125 
Attach cost = 912.4689331054688 
Deformation cost = 95.33280181884766
Time: 14.492053655907512
Iteration: 44 
Total energy = 1007.10791015625 
Attach cost = 914.8983764648438 
Deformation cost = 92.20954895019531
Time: 14.689171015052125
Iteration: 45 
Total energy = 1005.9403076171875 
Attach cost = 916.4203491210938 
Deformation cost = 89.5199

In [139]:
print("Optimised rotation center={center}".format(center=model.init_manifold[1].gd.detach().tolist()))

Optimised rotation center=[[15.49264144897461, 15.424398422241211]]


In [140]:
plt.plot(range(len(costs)), costs)
plt.show()

In [141]:
with torch.autograd.no_grad():
    deformed_image = model.compute_deformed(shoot_method, shoot_it)[0]

In [142]:
plt.subplot(1, 3, 1)
plt.imshow(source_image)

plt.subplot(1, 3, 2)
plt.imshow(deformed_image.numpy())

plt.subplot(1, 3, 3)
plt.imshow(target_image)
plt.show()